In [33]:
#;.pykx.disableJupyter()

PyKX now running in 'python' mode (default). All cells by default will be run as python code. 
Include '%%q' at the beginning of each cell to run as q code. 


In [34]:
# https://code.kx.com/pykx/3.0/examples/jupyter-integration.html#q-first-mode
import pykx as kx
kx.util.jupyter_qfirst_enable()

PyKX now running in 'jupyter_qfirst' mode. All cells by default will be run as q code. 
Include '%%py' at the beginning of each cell to run as python code. 


**Learning Outcomes**

* What is a namespace?
* How to identify a namespace
* How to create a namespace
* The difference between fully-qualified method and `\d` method
* Practical usage 
* What are internal functions?

# Introduction
[Namespaces](https://code.kx.com/q4m3/12_Workspace_Organization/#121-namespaces), also called contexts or directories, can be used to arrange code into logical groupings to make it easier to read and guard against name clashes between libraries - e.g.  multiple scripts that have a `saveTable` function defined.

<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:15px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>If you assign to a namespace within a function, this is another way of setting a variable globally outside function scope - e.g. <code>{.debug.catch:(x;y);x*y}</code> and can often be helpful when debugging! </i></p>

## What <i>is</i> a namespace? 
A namespace is really a dictionary! That's a simplification, but in terms of viewing (and accessing) a namespace they are structured like dictionaries with nulls for the first key/value pair.

For example, the [.Q](https://code.kx.com/q/ref/dotq/) namespace in kdb+/q is a built-in namespace that functions to perform commonly utilised tasks: 

In [5]:
.Q    //by calling .Q you can see the namespace dictionary

         | ::
ajf0     | k){[f;g;x;y;z]x,:();z:0!z;d:$[g;x_z;z];g:(:;^)f;f:(,;^)f;$[&/j:-1<..
k        | 4f
K        | 0Nd
host     | ![-12]
addr     | ![-13]
gc       | ![-20]
ts       | k){-34!(x;y)}
gz       | ![-35]
w        | k){`used`heap`peak`wmax`mmap`mphy`syms`symw!(."\\w"),."\\w 0"}
res      | `abs`acos`asin`atan`avg`bin`binr`cor`cos`cov`delete`dev`div`do`enl..
addmonths| k){("d"$m+y)+x-"d"$m:"m"$x}
Xf       | k){y 1:0xfe20,("x"$77+@x$()),13#0x00;(`$($y),"#")1:0x}
Cf       | k){y 1:0xfe20,("x"$77+@x$()),13#0x00;(`$($y),"#")1:0x}[`char]
f        | k){$[^y;"";y<0;"-",f[x;-y];y<1;1_f[x;10+y];9e15>j:"j"$y*prd x#10f;..
fmt      | k){$[x<#y:f[y;z];x#"*";(-x)$y]}
pykxld   | k){x:("#!"~2#*x)_x:-1!'x;+(1+*:'i;)@"\n"/:'x i:(&|1^\|0N 0 1@"/ "?..
ff       | k){$[&/(!+y)in f:!+x;x;x,'(f_y)(#x)#0N]}
fl       | k){$[98h=t:@x;+fl[+x;+y];99h=t;@[x,y;f;:;x[f]fl'y f@:&(f:!x)in!y];..
opt      | k){[o]x::$[#i:&o like"-[^0-9]*";i[0]#o;o];((`$1_*:)'o)!1_'o:i_o}
def      | k){x,((!y)#x){$[0h>@x;*:;:

Standard practice when using namespaces is to use dot (`.`) notation when accessing them, though standard dictionary accessing can be used too:

In [6]:
.Q.gc[]        //convention for using namespaces
.Q[`gc][]      //people don't use this syntax with namespaces, though it will work 
                    //we'll see later that we use this for setting context

0
0


[`.Q.gc[]`](https://code.kx.com/q/ref/dotq/#qgc-garbage-collect) returns the amount of memory that was returned to the OS. We will discuss this function more in detail when we are looking at the memory management module.  

<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:25px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>Namespaces are often created on initialization of processes to hold values which may be used within code. <br>  </i></p>

## Navigating Namespaces
The way in which a namespace is more than just a standard dictionary, is that we can "step into" our namespace view. This process of "stepping into" a namespace is referred to as changing the working context, and is best illustrated by an example:

In [ ]:
f: 1;          //defining f in our global view 
-1 "Value of f in global (`.) namespace:", string f; 

//changing into our namespace view 
\d .namespace  // \d lets us specify the namespace we want to work in 
f: 2;          //defining f within the .namespace view

-1 "Value of f in .namespace:", string f;              
\d .           //changing back to global view 

//in global view 
-1 "Value of f in global (`.) namespace:", string f;      


It's clearer to see this concept if viewing it from a terminal:

```
KDB+ 4.0 2020.03.17 Copyright (C) 1993-2020 Kx Systems
m64/ 12(16)core 32768MB fionnualacarr fionnualamacbook.local 192.168.1.6 EXPIRE 2021.03.30 fcarr@kx.com KOD #4169981

q)f: 1
q)\d .namespace       //stepping into .namespace
q.namespace)f:2       //defining f in .namespace
q.namespace)f
2
q.namespace)\d .      //stepping out of .namespace 
q)f
1
```

And similarly within functions the view namespace is what is referenced as the "global" namespace: 

In [ ]:
f: 1;          //defining f in our global view 
-1 "Value of 'function global {f}[]' f in global (`.) namespace:", string {f}[]; //a niladaic function referencing 'global'f
 
//changing into our namespace view 
\d .namespace  // \d lets us specify the namespace we want to work in 
f: 2;          //defining f within the .namespace view

-1 "Value of 'function global {f}[]' in .namespace:", string {f}[];              
\d .           //changing back to global view 

//in global view 
-1 "Value of 'function global {f}[]' f in global (`.) namespace:", string {f}[];      


# How to identify a namespace
Namespaces are identified by a leading dot in their names - e.g. `.space` references a namespace called space.

The following namespaces are already preloaded in your kdb+/q session by default:

|namespaces|contents|
|:----------|:--------|
|[.h](https://code.kx.com/q/ref/doth/)|Functions for converting files into various formats and for web-console display|
|[.j](https://code.kx.com/q/ref/dotj/) | Functions for converting between JSON and q dictionaries|
|[.Q](https://code.kx.com/q/ref/dotq/) | Utility functions|
|[.m](https://code.kx.com/q/ref/dotm/) (v 4.0+) | Used to allow explicit allocation to optane memory|
|[.z](https://code.kx.com/q/ref/dotz/) | System variables and functions, and hooks for callbacks|

<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:25px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> One character namespaces (like <code>.h</code>) are reserved exclusively for use by the KX software itself.<br>  </i></p>


We can find the list of namespaces that are present in the process by using the `key` function: 

In [7]:
key `       

`q`Q`h`j`o`pykx`comkxic`kurl`com_kx_log`s`p


##### Exercise 

Let's use what we know about namespaces to investigate and return all the variable names of the contents of the `.q` namespace. (i.e. the names of all functions and variable defined with that namespace)

In [8]:
key `.q   //key returns the list of all keys in a dictionary
                //for our namespace that is the list of all the variables defined within it!

``neg`not`null`string`reciprocal`floor`ceiling`signum`mod`xbar`xlog`and`or`ea..


Some of these should look familiar! 

Some functions in q are defined as expressions of a lower level language called `k` - when we start a `q` process our command prompt is prefaced by `q)` to indicate the namespace we are working in!

In [ ]:
//your answer here 

<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:10px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> In general, functions, variables and constants should only reside in the root namespace if they are accessed by external processes. If not, they should reside in a logical namespace. </i></p>

# How to create a namespace
There are two options for defining a namespace:

   1) The fully-qualified method (dot "." prefix)
   
   2) The `\d` (backslash d) method
   
   

## Fully Qualified Method (dot "." prefix)
When using this method, "globals" will be drawn from the context in which you are defining your function/variable. In order to have this recognised as a namespace we preface our functions/variable with a `.` (dot). 

In [37]:
//specifying the variable in the global context
f:15
.ns.f:10           //specifying a variable f of the same name in the namespace ns
.ns.function:{f}   //in ns also creating a function to return "f" from the creating context 
.ns.function[]     //what do you think this will return? 

15


Let's suppose we wanted to write some lambdas that we can use when working with cylinders. We can place all of these in a `.cylinder` namespace (notice the dot at the start):

In [38]:
PI:3.14159                     //defining pi as a global 
.cylinder.getArea:{PI*x*x*y}   //creating a function that will calculate the volume of our cylinder 
.cylinder.totalSurfaceArea:{2*PI*x*x+y} //creating a function that will calculate total surface area of our cylinder
.cylinder.dim.rad:3            //creating a nested namespace to store our dimensions
.cylinder.dim.len:10
.cylinder

                | ::
getArea         | {PI*x*x*y}
totalSurfaceArea| {2*PI*x*x+y}
dim             | ``rad`len!(::;3;10)


Looking at the `dim` entry in `.cylinder` we can see that it has created a general dictionary which has our cylinder dimensions stored within it. As a result of us using the fully defined method to describe the `.cylinder.getArea` function it has access to the root global `PI` and we can run it successfully: 

In [39]:
.cylinder.getArea[.cylinder.dim.rad;.cylinder.dim.len]

282.7431


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:5px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>Global Constants that are visible by all functions generally use the ALL_CAPS_UNDERSCORE convention e.g. TICK_PERIOD, LOG_LEVEL etc</i></p>


##### Functional notation 
As always, in kdb+/q we can perform updates functionally. To add new items to our namespace we can use the `@`(amend/apply) operator to directly update the namespace with new values we want to add: 

In [42]:
@[`.cylinder;`material;:;`steel]
.cylinder

.cylinder
                | ::
getArea         | {PI*x*x*y}
totalSurfaceArea| {2*PI*x*x+y}
dim             | ``rad`len!(::;3;10)
material        | `steel


**Be careful though! We can only functionally amend namespaces which have already been created.**

In [ ]:
@[`.myNewNamespace;`a;:;1] //this will throw an error

In [43]:
.myNewNamespace.a: 1  //direct assignment will create the namespace if it doesn't exist
.myNewNamespace

 | ::
a| 1


##### Deleting from a namespace 

If we want to delete a function or a variable from a namespace:

In [44]:
delete totalSurfaceArea from `.cylinder //deleting function as we don't use it
.cylinder

.cylinder
        | ::
getArea | {PI*x*x*y}
dim     | ``rad`len!(::;3;10)
material| `steel


## The `\d` method
The second method involves using `\d` to first move into the namespace specified.

In [45]:
\d .cylinder
getCircumference:{2*PI*x}           // and define new ones
getCircumference
\d .                                // move back to root, . is the root namespace
.cylinder                             // now we see .circle has changed

{2*PI*x}
                | ::
getArea         | {PI*x*x*y}
dim             | ``rad`len!(::;3;10)
material        | `steel
getCircumference| {2*PI*x}


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:10px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> In jupyter notebooks, we need to move in and out of the namespace in the same cell. In the terminal enviroment, it is much easier to see what is going on as our context is indicated on the right hand side prompt e.g. <code>q.ns)</code>. Try it out!</i></p>

What do you think will be returned by the below command? Why?

In [48]:
//.cylinder.getCircumference[.cylinder.dim.rad]   //calling our circumference example with the radius value

So what if we *want* to use the `\d` method, but still reference a root level global variable?  

In [49]:
\d .cylinder
getCircumference:{2*(`. `PI)*x}      //this change is a dicitonary lookup! 
getCircumference:{2*(`.[`PI])*x}     //restating more explicitly
\d .                                // move back to root, `. is the root namespace

.cylinder   

                | ::
getArea         | {PI*x*x*y}
dim             | ``rad`len!(::;3;10)
material        | `steel
getCircumference| {2*(`.[`PI])*x}


In [50]:
.cylinder.getCircumference[.cylinder.dim.rad]  

18.84954


The `\d` method is used when writing scripts and cannot be called within a function. 

##### Exercise

Create a global variable called `INT_RATE` with a value of 0.03 - this will be our interest rate. 

Create a namespace called `interest` that contains the values 
* `p`, value 300  (principal amount)
* `investTime`, value  5 (years)
* `simpleInterest`, a  function that will calculate the simple interest (interestRate x investTime x principal amount )

In [ ]:
//assuming simpleInterest function is niladic: 
INT_RATE:0.03      //Global so kept in the root context

\d .interest       //change to namespace we want to use
p: 300;            //principle amount - in namespace interest so don't use the leading .
investTime: 5      //investTime
\d .               //back to root 
.interest

//defining the simple interest function - since it needs to use INT_RATE which is in the root context 
   //we use the fully qualified method
.interest.simpleInterest:{INT_RATE*.interest.investTime*.interest.p}

//calling this 
.interest.simpleInterest[]

//changing our value of Interest will cause our function value to change since it's evaluated at run time
INT_RATE:0.04 
.interest.simpleInterest[]

In [52]:
//assuming simpleInterest function takes arguments
INT_RATE:0.03      //Global so kept in the root context

\d .interest       //change to namespace we want to use
p: 300;            //principle amount - in namespace interest so don't use the leading .
investTime: 5      //investTime
simpleInterest:{[p;r;investTime]p*r*investTime}   //defining the function in our namespace
\d .               //back to root 

.interest

//calling our function: 
.interest.simpleInterest[.interest.p;INT_RATE;.interest.investTime]

              | ::
principal     | 300
investmentTime| 5
simpleInterest| {[p;r;investTime]p*r*investTime}
p             | 300
investTime    | 5
45f


In [51]:
/write your code here
INT_RATE:0.03

.interest.principal:300
.interest.investmentTime:5
.interest.simpleInterest:{INT_RATE * x*y}
.interest.simpleInterest [.interest.principal;.interest.investmentTime]

45f


# Method differences and appropriate usage 
The main distinction between the two methods is the context available for functions at runtime - a simple rule-of-thumb to decide which to use is to ask the question "What context variables do I need (i.e. reference) in my code?".

 A secondary distinction is that we cannot use the `\d` method within a function:

In [ ]:
{system"\d myNewNameSpace";} []     //will throw an error!

## Which method to use


This is a matter of preference. If our namespace functions will frequently be accessing variables and functions in the root namespace, the fully-qualified method is best. On the contrary, if we want our code to be logically separated akin to other languages where sub-namespaces cannot naturally "see" above them, we may prefer the \d . method. 

There is no discernible performance impact one way or the other. 

## Worked Example 


### Simple
The following code is an example of using the `.log` namespace to store functions related to logging out messages. It is reproduced using the `\d` and fully defined method, to allow you to compare and contrast!

<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:5px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>Quick Recap- Do you remember what -1 and -2 are? </i></p>

If you previously attended the KX fundamentals course, we covered this when we were looking at string manipulation.
When displaying output, we can use:
* [Stdout](https://code.kx.com/q/basics/handles/) - 1&-1
* [Stderr](https://code.kx.com/q/basics/handles/#file-stdout-stderr) - 2&-2

In [53]:
-1"Hello World"; a:`other`stuff;

Hello World


In [10]:
-2 "Errors in Jupyter are indicated by a red band";2+2  //NB execution isn't stopped by writing to stderr

4


Errors in Jupyter are indicated by a red band


In [54]:
//\d method
\d .log 
info:{ -1 string[.z.p],"|",x;}   //information message 
err:{-2 string[.z.p],"|",x;}     //error message 
\d . 

.log.err["That's it?"]

2025.03.09D01:00:52.254406346|That's it?


In [55]:
//fully defined method
 
.log.info:{ -1 string[.z.p],"|",x;}   //information message 
.log.err:{-2 string[.z.p],"|",x;}     //error message 

.log.err["That's it?"]

2025.03.09D01:00:59.392307844|That's it?


There really isn't anything in what we're doing above to make this anything other than a preference choice!

### More complicated
Lets consider the potentially more realistic case where we might have a global variable setting our debug level (`DEBUG_LEVEL`) which can either be set to `DEBUG` or `NONE`. If we are set to `DEBUG` we want to print our information logs, otherwise we don't. 


In [56]:
//\d method
DEBUG_LEVEL:`DEBUG


\d .log 
info:{ if[`DEBUG~ `.[`DEBUG_LEVEL];-1 string[.z.p],"|",x];}   //information message 
                                                                //checking the root context explicitly for our setting
err:{-2 string[.z.p],"|",x;}                                  //error message 
\d . 

.log.info["Now we're moving on to the next step ..."]

2025.03.09D01:01:18.654837044|Now we're moving on to the next step ...


In [57]:
DEBUG_LEVEL:`NONE
.log.info["And the one after "]                               //note no output!

Fully defined method: 

In [58]:
//fully defined method
DEBUG_LEVEL:`DEBUG


.log.info:{ if[`DEBUG~DEBUG_LEVEL; -1 string[.z.p],"|",x];}   //information message 
                                                                //checking the root context already
.log.err:{-2 string[.z.p],"|",x;}                             //error message 


.log.info["Now we're moving on to the next step ..."]

2025.03.09D01:01:32.887349136|Now we're moving on to the next step ...


In [59]:
DEBUG_LEVEL:`NONE
.log.info["And the one after "]                               //note no output!

# Namespaces and Internal functions 

## What is an internal function?

An internal function is classified as the operator `!` with a negative integer as it's left argument. They are exposed infrastructure mainly for use by language implementers and may be redefined in subsequent releases. They also allow new language features to be tried on a provisional basis.
A list of all internal functions can be found [here](https://code.kx.com/q/basics/internal/).

## What do they have to do with Namespaces?

Generally the most important internal functions are added into namespaces over time. Once the internal functions are brought into namespaces, they become an integral part of the language and are guaranteed to be supported in later versions of kdb+. We would recommend using the namespace equivalent of any internal function if it exists. Below are the examples of the currently mapped internal functions:

|Internal Functions|Namespace Mapping|Description|
|:----------|:--------|:--------|
|-3!|[.Q.s1](https://code.kx.com/q/ref/dotq/#qs1-string-representation)|String Representation|
|-12!|[.Q.host](https://code.kx.com/q/ref/dotq/#qhost-hostname)|Hostname|
|-13!|[.Q.addr](https://code.kx.com/q/ref/dotq/#qaddr-ip-address)|IP address|
|-20!|[.Q.gc](https://code.kx.com/q/ref/dotq/#qgc-garbage-collect)|Garbage Collect|
|-29!|[.j.k](https://code.kx.com/q/ref/dotj/#jk-deserialize)|Deserialize|
|-31!|[.j.jd](https://code.kx.com/q/ref/dotj/#jjd-serialize-infinity)|Serialize Infinity|
|-32!|[.Q.btoa](https://code.kx.com/q/ref/dotq/#qbtoa-b64-encode)|b64 encode|
|-34!|[.Q.ts](https://code.kx.com/q/ref/dotq/#qts-time-and-space)|Time and Space|
|-35!|[.Q.gz](https://code.kx.com/q/ref/dotq/#qgz-gzip)|GZip|
|-37!|[.Q.prf0](https://code.kx.com/q/ref/dotq/#qprf0-code-profiler)|Code Profiler|
 

## Commonly used internal functions

The most popular internal function is the identity function ``ON!x``. 

In [60]:
0N!"hello" //printing 

hello
"hello"


It's widely used for debugging in which we'll discuss in depth later in the course. Here is an example that illustrates how useful it is when debugging:

In [61]:
fnShow:{sum show x<5} //using show to display the output
fnShow[3 6 2]

101b


The function doesn't calculate the summation of `101b` however if we use `0N!`:

In [62]:
fnDisplay:{sum 0N!x<5} //replacing show with 0N!
fnDisplay[3 6 2]

2i
101b


`0N!` can be added at any point in a function to print out a variable to the screen (equivalent to STDOUT).

Many are replaced by keywords or utilities, use the replacements where they are available!

In [63]:
show x:-15!"hello" 
show y:md5"hello"  //-15! was replaced by the keyword md5

x~y

1b
0x5d41402abc4b2a76b9719d911017c592
0x5d41402abc4b2a76b9719d911017c592


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:5px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>It is recommended to use replacement keyword (<a href="https://code.kx.com/q/ref/md5/">md5</a> in the above case) as the internal function might be redefined in future releases. </i></p>

Log files are replayed when a process shuts down or if we lost data. We can use an internal function ``-11!x`` in where x is a file handle. A simple example is found [here](https://code.kx.com/q/kb/replay-log/).

Later in the course, we discuss memory management that we can apply `-21!x` to a compressed file to see some compression stats. 

##### Quiz Time!
Try the Exercises to have a go at creating and using your own namespaces!

In [26]:
id:{(2#x)#1,x#0}
id 5

1 0 0 0 0
0 1 0 0 0
0 0 1 0 0
0 0 0 1 0
0 0 0 0 1


In [31]:
show id:[(2#5)#1,5#0]

1 0 0 0 0
0 1 0 0 0
0 0 1 0 0
0 0 0 1 0
0 0 0 0 1


In [28]:
{(2#x)#1,x#0}[5]

1 0 0 0 0
0 1 0 0 0
0 0 1 0 0
0 0 0 1 0
0 0 0 0 1


In [29]:
{(2#5)#1,5#0}

{(2#5)#1,5#0}


In [30]:
[(2#5)#1,5#0]

1 0 0 0 0
0 1 0 0 0
0 0 1 0 0
0 0 0 1 0
0 0 0 0 1


In [32]:
5 5#1,5#0

1 0 0 0 0
0 1 0 0 0
0 0 1 0 0
0 0 0 1 0
0 0 0 0 1
